In [1]:
from dragonfly.model import Model
from ladybug.location import Location
from dragonfly.building import Building
from dragonfly.story import Story
from dragonfly.room2d import Room2D
from dragonfly.windowparameter import SimpleWindowRatio
from honeybee_energy.lib.programtypes import office_program
from ladybug_geometry.geometry3d.pointvector import Point3D
from ladybug_geometry.geometry3d.face import Face3D

#https://github.com/ladybug-tools/honeybee-energy/blob/master/honeybee_energy/config.py
from honeybee_energy.config import folders

In [2]:
print(folders.energyplus_path)
print(folders.openstudio_path)
print(folders.energyplus_idd_path)
print(folders.honeybee_openstudio_gem_path)

/usr/local/openstudio-3.3.0/EnergyPlus
/usr/local/openstudio-3.3.0/bin
/usr/local/openstudio-3.3.0/EnergyPlus/Energy+.idd
None


In [3]:
# create the Building object
pts_1 = (Point3D(0, 0, 3), Point3D(0, 10, 3), Point3D(10, 10, 3), Point3D(10, 0, 3))
pts_2 = (Point3D(10, 0, 3), Point3D(10, 10, 3), Point3D(20, 10, 3), Point3D(20, 0, 3))
pts_3 = (Point3D(0, 10, 3), Point3D(0, 20, 3), Point3D(10, 20, 3), Point3D(10, 10, 3))
pts_4 = (Point3D(10, 10, 3), Point3D(10, 20, 3), Point3D(20, 20, 3), Point3D(20, 10, 3))
room2d_1 = Room2D('Office1', Face3D(pts_1), 3)
room2d_2 = Room2D('Office2', Face3D(pts_2), 3)
room2d_3 = Room2D('Office3', Face3D(pts_3), 3)
room2d_4 = Room2D('Office4', Face3D(pts_4), 3)
story = Story('OfficeFloor', [room2d_1, room2d_2, room2d_3, room2d_4])
story.solve_room_2d_adjacency(0.01)
story.set_outdoor_window_parameters(SimpleWindowRatio(0.4))
story.multiplier = 4
building = Building('OfficeBuilding', [story])

# assign energy properties
for room in story.room_2ds:
    room.properties.energy.program_type = office_program
    room.properties.energy.add_default_ideal_air()

# create the Model object
model = Model('NewDevelopment2', [building])

In [4]:
# create the dragonfly Model object
model = Model('NewDevelopment2', [building])
hb_models = model.to_honeybee('Building', use_multiplier=False, tolerance=0.01)
idfs = [hb_model.to.idf(hb_model) for hb_model in hb_models]

In [5]:
hb_model = hb_models[0]
hbjson_loc = hb_model.to_hbjson('test.hbjson')
hbjson_loc

'/root/simulation/test.hbjson'

In [10]:
# https://github.com/ladybug-tools/honeybee-energy/blob/master/honeybee_energy/run.py
from honeybee_energy.run import from_idf_osw, to_openstudio_osw, run_osw
osw = to_openstudio_osw('/mount/oswmodel', 
                        model_path=hbjson_loc,
                        epw_file='/mount/weather/USA_IL_Chicago.OHare.Intl.AP.725300_TMYx.epw',
#                         additional_measures=['reports'],
#                         report_units='ip',
                        schedule_directory='/mount/scheds')
print(osw)
run_out = run_osw(osw)
#from_idf_osw("basic.idf", model_path='/hbmodel', osw_directory='oswmodel')

/mount/oswmodel/workflow.osw


In [8]:
run_out

(None, None)

In [14]:
from honeybee_energy.writer import model_to_idf
with open("basic.idf", "w") as txt:
    txt.write(model_to_idf(hb_model))


/usr/local/openstudio-3.3.0/EnergyPlus
/usr/local/openstudio-3.3.0/EnergyPlus/Energy+.idd
